In [1]:
import os
import json

from utils import utils

NMR = "nmr-82"
VERSION = "1"
CONFIG_PATH = "./config/numerai/%s/%s-%s.json"%(NMR, NMR, VERSION)

with open(CONFIG_PATH) as f:
    conf = json.loads(f.read())

In [7]:
path = "%s/%s/%s"%(conf.get("data"),
                         conf.get("name"),
                         conf.get("evaluate").get("file"))
data = utils.load_data(path)
X = utils.get_features(data)

In [3]:
# Load models
models = conf.get("evaluate").get("models")
estimators = [utils.load_model(conf, model) for model in models]

In [8]:
# Evaluate single model
estimator = estimators[0]
_predictions = estimator.predict_proba(X)



In [16]:
import pandas as pd
 
predictions = pd.DataFrame()
predictions['id'] = data['id']
predictions['probability'] = _predictions[:,1]

pred_path = "%s/%s-%s.csv" % ("./predictions", conf.get("name"), conf.get("model_no"))
predictions.to_csv(pred_path,
                    sep=',', index=False)


In [18]:
from numerapi import NumerAPI

napi = NumerAPI(verbosity="info")
credentials = napi.login()

ul_succeeded = napi.upload_predictions(pred_path)
print("upload succeeded: " + str(ul_succeeded))


2017-11-18T14:11:30 INFO numerapi: logging in...


email: dhruv.nair@gmail.com
password: ········


HTTPError: 400 Client Error: Bad Request for url: https://api.numer.ai/sessions